In [1]:
import numpy as np # linear algebra
import pandas as pds # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.losses import BinaryCrossentropy
from keras.callbacks import Callback
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow import cast, int32

In [1]:
def get_dataset() :
    # Read the dataset
    df = pds.read_csv("Churn_Modelling.csv")
    df.drop(['RowNumber','CustomerId','Surname'],axis=1, inplace=True)
    df=pds.get_dummies(df, columns=['Geography','Gender'], drop_first=True)
    
    # Handling categorical values
    x=df.drop(columns=['Exited'])
    y=df['Exited']
    x_train, x_test, y_train, y_test=train_test_split(x,y, random_state=41, test_size=0.2)
    
    # Feature Scaling and test train split
    scaler = StandardScaler()
    x_train_scaled=scaler.fit_transform(x_train)
    x_test_scaled=scaler.transform(x_test)
    
    return  x_train_scaled, x_test_scaled, y_train.to_numpy(), y_test.to_numpy()

In [2]:
def get_model() : 
    model=Sequential()
    model.add(Dense(11, activation='relu', input_dim=11))
    model.add(Dense(11, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    return model

In [4]:
def get_shape(model):
    weights_layer = model.get_weights()
    shapes = []
    for weights in weights_layer:
        shapes.append(weights.shape)
    return shapes

In [5]:
def set_shape(weights, shapes):
    new_weights = []
    index = 0
    for shape in shapes:
        if (len(shape) > 1):
            n_nodes = np.prod(shape) + index
        else:
            n_nodes = shape[0] + index
        tmp = np.array(weights[index:n_nodes]).reshape(shape)
        new_weights.append(tmp)
        index = n_nodes
    return new_weights

In [2]:
def get_con_mat(model,X,Y):
    y_pred = model.predict(X)
    y_pred_binary = cast(y_pred > 0.5, dtype= int32).numpy()
    con_1 = confusion_matrix(Y, y_pred_binary)
    plt.style.use('seaborn-deep')
    plt.figure(figsize=(5,5))
    sns.heatmap(con_1,annot=True,annot_kws={'size': 15},linewidths=0.5,fmt="d",cmap="Blues")
    plt.title('Churn confusion matrix\n',fontweight='bold',fontsize=15)
    plt.show()

In [7]:
class LossHistory(Callback):
    def on_train_begin(self, logs=None):
        self.losses = []

    def on_batch_end(self, batch, logs=None):
        if len(self.losses) == 0:
            self.losses.append(logs['loss'])
        else:
            if logs['loss'] < self.losses[-1]:
                self.losses.append(logs['loss'])
            else:
                 self.losses.append(self.losses[-1])